<a href="https://colab.research.google.com/github/bazsi04/Szakdolgozat/blob/main/Algoritmusok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importálás

In [ ]:
from heapq import heappush, heappop
import numpy as np
from itertools import combinations
import random
import networkx as nx
import math
import time
import matplotlib.pyplot as plt

### Algoritmusok

Nem-negatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
def nemneg_mon(input):
    n, adj_list, s = input
    V = list(range(1, n+1))
    P = []
    S = {s}
    p_v = {s: s}
    p_e = {}
    R = {s: 0}
    K = {}


    h = [0 for u in V]
    degree = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}

    for v, cost in adj_list[s]:
        heappush(P, (cost, s, v, cost))
        K[s,v] = cost
        p_e[s,v] = s
        R[v] = cost
        p_v[v] = (s,v)


    while P:
        K_uv, u, v, c_uv = heappop(P)
        if h[v-1] > degree[v]:
            continue


        for i in range(h[v-1], degree[v]):
            t, c_vt = F[v][i]
            if c_vt >= c_uv:
                break

            K_total = K_uv + c_vt
            if (v,t) not in K or K_total < K[v,t]:
                K[v,t] = K_total
                p_e[v,t] = (u,v)
                heappush(P, (K_total, v, t, c_vt))
            if t not in R or K_total < R[t]:
                R[t] = K_total
                p_v[t] = (v,t)

            h[v-1] += 1

    return R, p_v, p_e

Konzervatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
def konz_mon(input):
    n, adj_list, s = input
    V = list(range(1, n + 1))

    degree = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}  # élsúly szerint rendezve
    p_e = {}
    R = {s: 0}
    p_v = {s: s}
    K = {}
    p = {}
    c = {}

    # Kezdeti értékek
    p[s] = 0
    R[s] = 0
    for u in V:
        for v, cost in adj_list[u]:
            c[u,v] = cost
            K[u, v] = float('inf')
    for v, cost in adj_list[s]:
        p[s, v] = cost
        K[s, v] = cost
        R[v] = cost


    for _ in range(n - 1):
        for u in V:
            h = [0 for u in V]

        Q = sorted(K.keys(), key=lambda edge: K[edge]) #K alapján növekvő sorba rendezve
        for (u, v) in Q:
            if h[v-1] > degree[v]:
                continue

            for i in range(h[v-1], degree[v]):
                t, c_vt = F[v][i]
                if c_vt >= c[u,v]:
                    break

                if K[(u, v)] + c_vt < K[(v, t)]:
                    K[(v, t)] = K[(u, v)] + c_vt
                    p[(v, t)] = (u, v)
                    if t not in R or K[(v, t)] < R[t]:
                        R[t] = K[(v, t)]
                        p_v[t] = (v, t)

                h[v-1] += 1

    return R, p_v, p_e

Nem-negatív csúcssúlyozású minimális út

In [ ]:
def csucs_min(input):
    n, adj_list, s, w = input
    V = list(range(1, n+1))
    P = []
    M = set(V) - {s}
    p = {s: s}
    K = {s: 0}

    heappush(P, (w[s], s))
    while P:
        K_u, u = heappop(P)

        for v in adj_list[u]:
            if v in M:
                K[v] = K_u + w[v]
                p[v] = u
                heappush(P, (K[v], v))
                M.remove(v)

    return K, p

Legfeljebb k élű minimális út

In [ ]:
def k_el_min(input):
    n, adj_list, s, k = input
    K = np.full((k+1, n), np.inf)
    p = np.full((k+1, n), None)

    K[0][s-1] = 0
    p[0][s-1] = s
    for j in range(1, k+1):
        for u in range(1,n+1):
            K[j][u-1] = K[j-1][u-1]
            p[j][u-1] = p[j-1][u-1]
        for u in range(1,n+1):
            for v, cost in adj_list[u]:
                if K[j][v-1] > K[j-1][u-1] + cost:
                    K[j][v-1] = K[j-1][u-1] + cost
                    p[j][v-1] = u

    return K, p

Pontosan k-hosszú út

In [ ]:
def coloring(n, adj_list, k, seed = None):
    if seed is not None:
        random.seed(seed)
    V = list(range(1, n+1))
    node_colors = {v: random.randint(0, k-1) for v in V}
    new_adj_list = {v: [] for v in V}


    for u in adj_list:
        for v, w in adj_list[u]:
            new_adj_list[u].append((v,w,node_colors[v]))


    return node_colors, new_adj_list

In [ ]:
def colorcoding(input, seed = None):
    n, adj_list, s, k = input

    f = {}
    p = {}
    colors = set(range(0,k))
    _, adj_list = coloring(n, adj_list, k, seed)

    rev_lst = {u: [] for u in adj_list}

    for u in adj_list:
        for v, weight, color in adj_list[u]:
            rev_lst[v].append((u, weight, color))

    # Színek részhalmazai elemszám szerint növekvő sorrendben
    P = [frozenset(comb) for i in range(1, k+1) for comb in combinations(colors, i)]


    for p_set in P:
        for u in adj_list:
            if not adj_list[u]:
                continue
            for v, _, edge_color in adj_list[u]:
                f[p_set, (u, v)] = False

            if u == s and edge_color in p_set:
                f[p_set, (u, v)] = True
                p[p_set, (u, v)] = s

    for p_set in P[k:]:
        for u in adj_list:
            for (v, w, edge_color) in adj_list[u]:
                f[p_set, (u, v)] = False
                if edge_color in p_set:
                    for (t, ww, _) in rev_lst[u]:
                        if f.get((p_set - {edge_color}, (t, u)), True) and ww < w and v != s:
                            f[p_set, (u, v)] = True
                            p[p_set, (u, v)] = (t, u)


    f_v = {}
    p_v = {}

    for u in adj_list:
        for v, _, _ in adj_list[u]:
            if f.get((P[-1], (u, v)), True):
                f_v[v] = True
                p_v[v] = (u, v)

    return f_v, p_v, f, p